## Phase 1 Project Data Cleaning / EDA

## Business Problem

This is the problem our project is specifying:

"Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase."

In this EDA, I do the following:

- Clean the Aviation_Data.csv dataset by:
    - Removing irrelevant measures
    - Removing duplicated records
    - Carefully removing or replacing null values
- Run a few different analyses to determine the lowest risk aircraft

## Importing the Data

In [1]:
! ls data

Aviation_Data.csv
Aviation_Data_Cleaned.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

We had an import issue that Nick solved by specifying the data type for columns 6, 7, and 28.

In [3]:
with open('data/Aviation_Data.csv', encoding='utf8') as f:
    dtypes = {'Column6Name': 'str', 'Column7Name': 'str', 'Column28Name': 'str'}
    df = pd.read_csv(f, dtype=dtypes, low_memory=False)

In [4]:
df.head(2)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

Okay, so we've got 90,348 records and 31 columns. Let's clean this up a bit.

## Cleaning the Data

Keeping / Removing Columns for These Reasons

Keeping

- Event ID and Accident Number: keeping for now to screen for duplicates in next step*
- Event.Date: might be useful
- Location and Country: might be useful
- Injury Severity, Aircraft Damage: will be useful
- Aircraft Category: Shows many records not involving airplanes, which we can remove later*
- Make and Model: will be useful
- Amateur Built: can probably use to remove from data. About 8.5k out of 90 are listed as amateur built*
- Number of Engines: will need further investigation. Some show 0 engines, most show one. I would imagine we would only be looking at 2+ engine planes for enterprise use*
- Engine Type: useful
- FAR Description: Stands for Federal Aviation Regulation description. Might be useful, so keep for now.*
- Purpose of Flight: useful for determining cause of accident
- Total Injuries columns: useful
- Weather Conditions: will help determine possible cause
- Report Status: May be helpful for us

*Return to these measures for further cleaning

Removing

- Investigation Type: upon doing a .values_count(), we see it is not useful
- Latitude and Longitude: mostly null, also redundant since we have Location (city, state)
- Airport.Code and Airport Name: about half null, also irrelevant to determining safety
- Registration Number: irrelevant
- Schedule: Mostly null, also irrelevant
- Air Carrier: mostly null, aldo irrelevant
- Broad Phase of Flight: Enough null values for a categorical data point that we should exclude
- Publication Date: irrelevant when data was published

In [6]:
df.drop(['Investigation.Type','Latitude', 'Longitude', 'Airport.Code', 
         'Airport.Name', 'Registration.Number', 'Schedule', 'Air.carrier', 
         'Broad.phase.of.flight', 'Publication.Date'], axis=1, inplace=True)

In [7]:
df.head(2)

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Aircraft.Category,Make,Model,...,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
0,20001218X45444,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,Fatal(2),Destroyed,NaN,Stinson,108-3,...,1.0,Reciprocating,NaN,Personal,2.0,0.0,0.0,0.0,UNK,Probable Cause
1,20001218X45447,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,Fatal(4),Destroyed,NaN,Piper,PA24-180,...,1.0,Reciprocating,NaN,Personal,4.0,0.0,0.0,0.0,UNK,Probable Cause


In [8]:
df[df['Event.Id'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 64030 to 90097
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                0 non-null      object 
 1   Accident.Number         0 non-null      object 
 2   Event.Date              0 non-null      object 
 3   Location                0 non-null      object 
 4   Country                 0 non-null      object 
 5   Injury.Severity         0 non-null      object 
 6   Aircraft.damage         0 non-null      object 
 7   Aircraft.Category       0 non-null      object 
 8   Make                    0 non-null      object 
 9   Model                   0 non-null      object 
 10  Amateur.Built           0 non-null      object 
 11  Number.of.Engines       0 non-null      float64
 12  Engine.Type             0 non-null      object 
 13  FAR.Description         0 non-null      object 
 14  Purpose.of.flight       0 non-null 

Getting all null values in rows where Event ID is null. Let's drop those records

In [9]:
df.dropna(subset=['Event.Id'], inplace=True)

Time to look for duplicates

In [10]:
# New df containing duplicated Event ID's

# df_duplicates1 = df[df.duplicated(subset=['Event.Id'], keep=False) == True].sort_values('Event.Id')

# df_duplicates1

Looks like we found some misentries. The makes and models of these aircraft are disagreeing for seemingly the same accidents, which has created these duplicate records. Unfortunately, it's impossible for us to tell which record is accurate. To avoid bad data messing with our results, we will need to exclude ALL of these records.

In [11]:
# df.drop_duplicates(subset=['Event.Id'], keep=False, inplace=True)

In [12]:
# Checking our work

# df.duplicated(subset=['Event.Id']).value_counts()

In [13]:
# Let's check along the Accident Number column

# df.duplicated(subset=['Accident.Number']).value_counts()

Looks good. For cleanliness, let's drop the Accident Number column. We can keep the Event ID for indexing and to allow others to check for duplicates.

In [14]:
# df.drop(['Accident.Number'], axis=1, inplace=True)

Now let's look at Aircraft Category and exclude any non-airplane records

In [15]:
# These are the records we need to drop

df[(df['Aircraft.Category'] != 'Airplane') & (df['Aircraft.Category'].isna() == False)]

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Aircraft.Category,Make,Model,...,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
16,20020917X01962,DEN82DTM08,1982-01-02,"MIDWAY, UT",United States,Non-Fatal,Destroyed,Helicopter,Enstrom,280C,...,1.0,Reciprocating,Part 91: General Aviation,Personal,0.0,0.0,0.0,1.0,IMC,Probable Cause
19,20020917X02339,MIA82DA028,1982-01-02,"MIAMI, FL",United States,Non-Fatal,Substantial,Helicopter,Smith,WCS-222 (BELL 47G),...,1.0,Reciprocating,Part 91: General Aviation,Personal,0.0,0.0,0.0,2.0,VMC,Probable Cause
22,20020917X01657,ATL82DA027,1982-01-02,"CHAMBLEE, GA",United States,Non-Fatal,Substantial,Helicopter,Bell,206L-1,...,1.0,Turbo Shaft,Part 135: Air Taxi & Commuter,Unknown,0.0,0.0,0.0,1.0,VMC,Probable Cause
46,20020917X02157,LAX82DA039,1982-01-06,"MAMMOTH LAKES, CA",United States,Non-Fatal,Substantial,Helicopter,Aerospatiale,SA-316B,...,1.0,Turbo Shaft,Part 91: General Aviation,Business,0.0,0.0,0.0,6.0,VMC,Probable Cause
62,20020917X02247,LAX82DVG13,1982-01-09,"CALISTOGA, CA",United States,Non-Fatal,Substantial,Glider,Schleicher,ASW 20,...,0.0,Unknown,Part 91: General Aviation,Personal,0.0,0.0,0.0,1.0,VMC,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90301,20221204106407,ERA23FA078,2022-12-04,"Beverly, MA",United States,Fatal,Substantial,Gyrocraft,ROTORSPORT UK LTD,CAVALON,...,1.0,NaN,091,Personal,1.0,0.0,0.0,0.0,VMC,NaN
90313,20221209106435,WPR23LA061,2022-12-07,"Waimea, HI",United States,Minor,Substantial,Helicopter,EUROCOPTER,EC 130 B4,...,1.0,NaN,135,NaN,0.0,1.0,0.0,6.0,NaN,NaN
90315,20221212106442,WPR23LA063,2022-12-08,"La Sal, UT",United States,Non-Fatal,Substantial,Helicopter,HUGHES,369D,...,1.0,NaN,091,Other Work Use,0.0,0.0,0.0,2.0,VMC,NaN
90322,20221213106449,GAA22WA311,2022-12-11,"Kildare,",Ireland,NaN,Destroyed,Helicopter,ROBINSON HELICOPTER COMPANY,R44 II,...,1.0,NaN,NUSN,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [16]:
# Get their indexes and drop

nonplane_indexes = df[(df['Aircraft.Category'] != 'Airplane') 
                      & (df['Aircraft.Category'].isna() == False)].index

df.drop(index=nonplane_indexes, inplace=True)

In [17]:
# Now let's drop that column

df.drop(['Aircraft.Category'], axis=1, inplace=True)

Let's look further into Engine Counts. We can probably drop all records for single engine aircraft, as our company probably will not be using prop planes. But first - what's going on with the 0 engines?

In [18]:
df['Number.of.Engines'].value_counts()

1.0    66360
2.0    10840
0.0      728
3.0      482
4.0      426
6.0        1
8.0        1
Name: Number.of.Engines, dtype: int64

In [19]:
df[df['Number.of.Engines'] == 0]

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
1505,20020917X03932,NYC82DA121,1982-06-09,"SHREWSBURY, PA",United States,Non-Fatal,Substantial,Scheicher,K8B,No,0.0,Unknown,NaN,Personal,0.0,0.0,0.0,1.0,VMC,Probable Cause
3606,20001214X42064,MKC83LA051,1983-01-02,"INDIANOLA, IA",United States,Non-Fatal,NaN,Balloon Works,FIREFLY 7B,No,0.0,Unknown,NaN,Personal,0.0,1.0,0.0,1.0,VMC,Probable Cause
3659,20001214X42066,MKC83LA053,1983-01-08,"GREENWOOD, MO",United States,Non-Fatal,Substantial,Balloon Works,FIRE FLY 7-B,No,0.0,Unknown,NaN,Instructional,0.0,0.0,0.0,2.0,VMC,Probable Cause
3951,20001214X42143,ATL83LA123,1983-02-21,"WOODBINE, MD",United States,Non-Fatal,Substantial,Scheibe Flugzeugbau,L SPATZ-55,No,0.0,Unknown,NaN,Personal,0.0,0.0,0.0,1.0,VMC,Probable Cause
4093,20001214X42553,NYC83LA076,1983-03-12,"BUENA VISTA, PA",United States,Non-Fatal,Substantial,Burkhart Grob,G10Z ASTIR CS,No,0.0,Unknown,NaN,Personal,0.0,0.0,0.0,1.0,VMC,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63301,20070907X01321,DEN07LA152,2007-09-02,"HUTCHINSON, KS",United States,Non-Fatal,Substantial,Slingsby,Swallow Type T.45,No,0.0,NaN,NaN,Glider Tow,NaN,1.0,NaN,NaN,VMC,Probable Cause
63538,20071029X01673,DEN08LA004,2007-10-08,"ALBUQUERQUE, NM",United States,Fatal(1),Substantial,Aerostar,S-66A,No,0.0,NaN,NaN,Personal,1.0,2.0,NaN,2.0,VMC,Probable Cause
63634,20071030X01689,DEN08LA017,2007-10-26,"Salida, CO",United States,Fatal(1),Destroyed,Schempp-hirth,Ventus B/16.6,No,0.0,NaN,NaN,Personal,1.0,NaN,NaN,NaN,VMC,Probable Cause
64890,20080827X01334,CHI08CA202,2008-07-05,"Beloit, WI",United States,Non-Fatal,Substantial,AB Sportine Aviacija,Genesis 2,No,0.0,NaN,091,Personal,0.0,0.0,0.0,1.0,VMC,The inadequate visual lookout by the driver of...


With a bit of googling, we find that these are gliders and balloons. Exclude!

In [20]:
# Get their indexes and drop

engines_1and0_indexes = df[(df['Number.of.Engines'] == 0) 
                      | (df['Number.of.Engines'] == 1)].index

df.drop(index=engines_1and0_indexes, inplace=True)

Now let's drop any amateur-built planes. We certainly are not in the market for those.

In [21]:
# Get their indexes and drop

amateur_indexes = df[(df['Amateur.Built'] == 'Yes')].index

df.drop(index=amateur_indexes, inplace=True)

In [22]:
# Now let's drop that column

df.drop(['Amateur.Built'], axis=1, inplace=True)

Where Injury Severity is listed as 'Incident', Aircraft Damage is null or Minor and Injuries are null, zero, or low.

Where Injury Severity is null, Injuries are null, zero, or low.

Let's drop these records.

In [23]:
df[(df['Injury.Severity'] == 'Incident') | (df['Injury.Severity'].isna())]

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
79,20020917X01897,CHI82IA026,1982-01-12,"CHICAGO, IL",United States,Incident,NaN,Lockheed,L-1011,3.0,Turbo Fan,Part 121: Air Carrier,Unknown,0.0,0.0,0.0,149.0,UNK,Probable Cause
80,20020917X01765,ATL82IA034,1982-01-12,"CLARKSBURG, WV",United States,Incident,Minor,Embraer,EMB-110P1,2.0,Turbo Prop,Part 135: Air Taxi & Commuter,Unknown,0.0,0.0,0.0,2.0,VMC,Probable Cause
119,20020917X01766,ATL82IA038,1982-01-19,"WASHINGTON, DC",United States,Incident,Minor,De Havilland,DHC-6-300,2.0,Turbo Prop,Part 91: General Aviation,Ferry,0.0,0.0,0.0,1.0,IMC,Probable Cause
131,20020917X02334,LAX82IA044,1982-01-20,"SAN JOSE, CA",United States,Incident,Minor,Piper,PA-31-350,2.0,Reciprocating,Part 91: General Aviation,Executive/corporate,0.0,0.0,0.0,2.0,VMC,Probable Cause
149,20020917X01767,ATL82IA041,1982-01-22,"LOUISVILLE, KY",United States,Incident,NaN,Dassault/sud,FALCON 20,2.0,Turbo Fan,Part 121: Air Carrier,Unknown,0.0,0.0,0.0,2.0,VMC,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90321,20221212106440,ERA23LA084,2022-12-11,"Fox Island, NY",United States,NaN,NaN,ROBINSON HELICOPTER,R22 BETA,NaN,NaN,091,Instructional,0.0,0.0,0.0,0.0,NaN,NaN
90333,20221215106462,CEN23LA064,2022-12-15,"Patterson, LA",United States,NaN,NaN,BELL,206-L4,NaN,NaN,135,NaN,0.0,0.0,0.0,0.0,NaN,NaN
90338,20221219106472,DCA23LA096,2022-12-18,"Kahului, HI",United States,NaN,NaN,AIRBUS,A330-243,NaN,NaN,121,NaN,0.0,0.0,0.0,0.0,NaN,NaN
90344,20221227106494,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,BELLANCA,7ECA,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [24]:
# Get their indexes and drop

incident_null_indexes = df[(df['Injury.Severity'] == 'Incident') | (df['Injury.Severity'].isna())].index

df.drop(index=incident_null_indexes, inplace=True)

In [25]:
null_make_model_index = df[(df['Make'].isna()) | (df['Model'].isna())].index

df.drop(index=null_make_model_index, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14028 entries, 4 to 90347
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                14028 non-null  object 
 1   Accident.Number         14028 non-null  object 
 2   Event.Date              14028 non-null  object 
 3   Location                14004 non-null  object 
 4   Country                 13961 non-null  object 
 5   Injury.Severity         14028 non-null  object 
 6   Aircraft.damage         12671 non-null  object 
 7   Make                    14028 non-null  object 
 8   Model                   14028 non-null  object 
 9   Number.of.Engines       9923 non-null   float64
 10  Engine.Type             11048 non-null  object 
 11  FAR.Description         5177 non-null   object 
 12  Purpose.of.flight       10720 non-null  object 
 13  Total.Fatal.Injuries    12370 non-null  float64
 14  Total.Serious.Injuries  11888 non-null

Now with cleaner data set I can wrap my head around some potential questions. In general, I'd like to know which Makes, Models, Engine Counts, and Engine Types are most/least involved in accidents, which I would like to measure by Injury Severity, Aircraft Damage, and Fatal and Serious Injury count.

Before going further, I'd like to try to fill in some null values for Engine Count and Engine Type by making a dictionary of existing counts and types by Model. Let's see if it works.

In [27]:
# Starting number

df['Number.of.Engines'].count()

9923

In [28]:
# Creating a dictionary using Model as keys and Number of Engines (non-null) as values

engine_count_dict = df[(df['Number.of.Engines'].isna() == 
                        False)].set_index('Model')['Number.of.Engines'].to_dict()

In [29]:
# Using this dictionary to fill in some null values in Number of Engines

df['Number.of.Engines'] = df['Number.of.Engines'].fillna(df['Model'].map(engine_count_dict))

In [30]:
# Ending number. Got another 1500!

df['Number.of.Engines'].count()

11457

In [31]:
# Starting number

df['Engine.Type'].count()

11048

In [32]:
# {Model: Engine Type}, then use to fill in nulls. Got another 2000!

engine_type_dict = df[(df['Engine.Type'].isna() == 
                        False)].set_index('Model')['Engine.Type'].to_dict()

df['Engine.Type'] = df['Engine.Type'].fillna(df['Model'].map(engine_type_dict))

df['Engine.Type'].count()

13171

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14028 entries, 4 to 90347
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                14028 non-null  object 
 1   Accident.Number         14028 non-null  object 
 2   Event.Date              14028 non-null  object 
 3   Location                14004 non-null  object 
 4   Country                 13961 non-null  object 
 5   Injury.Severity         14028 non-null  object 
 6   Aircraft.damage         12671 non-null  object 
 7   Make                    14028 non-null  object 
 8   Model                   14028 non-null  object 
 9   Number.of.Engines       11457 non-null  float64
 10  Engine.Type             13171 non-null  object 
 11  FAR.Description         5177 non-null   object 
 12  Purpose.of.flight       10720 non-null  object 
 13  Total.Fatal.Injuries    12370 non-null  float64
 14  Total.Serious.Injuries  11888 non-null

In [34]:
df[(df['Injury.Severity'] != 'Non-Fatal') & (df['Total.Fatal.Injuries'].isna())]

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
41240,20010711X01373,DCA97WA025,1997-02-08,"NURNBERG/GERF, Germany",Germany,Unavailable,NaN,Boeing,B-737-300,2.0,Turbo Fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign
41339,20010711X01374,DCA97WA033,1997-03-07,"MEDELLIN, Central African Republic",Central African Republic,Unavailable,NaN,Cessna,Citation 500,2.0,Turbo Fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign
41656,20001208X07697,DCA97WA043,1997-04-27,"JINAN, CHINA",China,Unavailable,Substantial,Mcdonnell Douglas,MD-82,2.0,Unknown,"Non-U.S., Commercial",Unknown,NaN,NaN,NaN,NaN,UNK,Foreign
42287,20001208X08352,DCA97WA054,1997-07-21,"SOUTHERN THAILA, Thailand",Thailand,Unavailable,Destroyed,Learjet,LR-31,NaN,Unknown,Part 129: Foreign,Unknown,NaN,NaN,NaN,NaN,UNK,Foreign
42847,20001208X08992,DCA98WA082,1997-10-09,"HARBIN, CHINA",China,Unavailable,Substantial,Cessna,650-0220,NaN,Unknown,"Non-U.S., Non-Commercial",Unknown,NaN,NaN,NaN,NaN,UNK,Foreign
43692,20010711X01369,DCA98WA033,1998-04-12,"ALMATY, Kazakhstan",Kazakhstan,Unavailable,NaN,Boeing,B-737-200,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign
44014,20001211X10045,DCA98WA048,1998-05-29,"BOMBAY, INDIA",India,Unavailable,Substantial,Boeing,B-737-500,NaN,Unknown,Part 129: Foreign,Unknown,NaN,NaN,NaN,NaN,UNK,Foreign
44608,20001211X10826,DCA98WA079,1998-08-05,"SEOUL, KOREA, REPUBLIC OF","Korea, Republic Of",Unavailable,Substantial,Boeing,747-400,4.0,Unknown,"Non-U.S., Commercial",Unknown,NaN,NaN,NaN,NaN,IMC,Foreign
45560,20001204X00045,DCA99WA035,1999-01-28,"NAPLES, ITALY",Italy,Unavailable,Substantial,Mcdonnell Douglas,MD-82,2.0,Unknown,Part 129: Foreign,Unknown,NaN,NaN,NaN,NaN,UNK,Foreign
45711,20001205X00299,DCA99WA050,1999-03-04,"BIARRITZ, FRANCE",France,Unavailable,Substantial,Boeing,737-228,NaN,Unknown,Part 129: Foreign,Unknown,NaN,NaN,NaN,NaN,IMC,Foreign


In [35]:
df['Report.Status'].value_counts()

Probable Cause                                                                                                                                                                                                                                                                                                                    8027
Foreign                                                                                                                                                                                                                                                                                                                           1393
<br /><br />                                                                                                                                                                                                                                                                                                                       101
Factual            

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14028 entries, 4 to 90347
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                14028 non-null  object 
 1   Accident.Number         14028 non-null  object 
 2   Event.Date              14028 non-null  object 
 3   Location                14004 non-null  object 
 4   Country                 13961 non-null  object 
 5   Injury.Severity         14028 non-null  object 
 6   Aircraft.damage         12671 non-null  object 
 7   Make                    14028 non-null  object 
 8   Model                   14028 non-null  object 
 9   Number.of.Engines       11457 non-null  float64
 10  Engine.Type             13171 non-null  object 
 11  FAR.Description         5177 non-null   object 
 12  Purpose.of.flight       10720 non-null  object 
 13  Total.Fatal.Injuries    12370 non-null  float64
 14  Total.Serious.Injuries  11888 non-null

Let's clean up the makes by combining anything that is supposed to be the same company.

In [37]:
df['Make'] = df['Make'].str.title()

In [38]:
df['Make'].value_counts().head(20)

Cessna               3084
Piper                2487
Beech                2104
Boeing               1271
Mcdonnell Douglas     257
Aero Commander        202
Douglas               188
Bell                  174
Embraer               168
Schweizer             165
Airbus                144
Swearingen            143
Mitsubishi            134
Learjet               121
De Havilland          121
Grumman               100
Rockwell               90
Sikorsky               86
Balloon Works          86
Airbus Industrie       82
Name: Make, dtype: int64

In [39]:
df['Event.Date'] = pd.to_datetime(df['Event.Date'])

In [40]:
df['day'] = df['Event.Date'].dt.day
df['month'] = df['Event.Date'].dt.month
df['year'] = df['Event.Date'].dt.year

In [41]:
df.head(2)

,Event.Id,Accident.Number,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Number.of.Engines,...,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status,day,month,year
4,20041105X01764,CHI79FA064,1979-08-02,"Canton, OH",United States,Fatal(1),Destroyed,Cessna,501,2.0,...,Personal,1.0,2.0,NaN,0.0,VMC,Probable Cause,2,8,1979
5,20170710X52551,NYC79AA106,1979-09-17,"BOSTON, MA",United States,Non-Fatal,Substantial,Mcdonnell Douglas,DC9,2.0,...,NaN,NaN,NaN,1.0,44.0,VMC,Probable Cause,17,9,1979


In [42]:
df[(df['Make'] == 'Douglas')]['year'].value_counts().index.min()

1982

In [43]:
# Event year for the first Douglas make was 1982, which is after Douglas merged with McDonnell. 
# Combine: McDonnell Douglas, Douglas, Mcdonald Douglas

df['Make'] = df['Make'].replace(['Douglas', 'Mcdonald Douglas', 'Mcdonnell Douglas Aircraft Co', 'Mcdonnell Douglas Corporation', 'Mcdonnell-Douglas'], 'Mcdonnell Douglas')

In [44]:
# same thing for airbus and airbus industrie - these are the same

df['Make'] = df['Make'].replace('Airbus Industrie', 'Airbus')

In [45]:
# same thing for gulfstream and Gulfstream Aerospace - these are the same

df['Make'] = df['Make'].replace(['Gulfstream', 'Gulfstream Aerospace Corp.', 'Gulfstream Aerospace Corp', 'Gulfstream Am Corp Comm Div', 'Gulfstream American', 'Gulfstream American Corp.', 'Gulfstream-Schweizer'], 'Gulfstream Aerospace')

In [46]:
# 737 is just a boeing. replace with boeing.
df['Make'] = df['Make'].replace('737', 'Boeing')
df['Make'] = df['Make'].replace(["Aerostar International", 'Aerostar International Inc', 'Aerostar International Inc.', 'Aerostar International, Inc.'], 'Aerostar')
df['Make'] = df['Make'].replace('Air Tractor Inc', 'Air Tractor')
df['Make'] = df['Make'].replace('American Blimp Corp.', 'American Blimp')
df['Make'] = df['Make'].replace('American Champion Aircraft', 'American Champion')
df['Make'] = df['Make'].replace('Agusta-Bell', 'Agusta')
df['Make'] = df['Make'].replace('As+ Ltd.', 'As+ Ltd')
df['Make'] = df['Make'].replace('Avian Balloon', 'Avian')
df['Make'] = df['Make'].replace(['Aviat Aircraft Inc', 'Aviat Inc'], 'Aviat')
df['Make'] = df['Make'].replace(['Bae Systems (Operations) Limit', 'Bae'], 'Bae Systems')
df['Make'] = df['Make'].replace("Beagle", 'Beagle Aircraft')
df['Make'] = df['Make'].replace(['Beech Aircraft', 'Beech Aircraft Co.', 'Beech Aircraft Corporation', 'Beechcraft'], 'Beech')
df['Make'] = df['Make'].replace(['Boeing - Canada (De Havilland)', 'Boeing Company', 'Boeing Of Canada/Dehav Div', 'Boeing Vertol'], 'Boeing')
df['Make'] = df['Make'].replace(['Bombardier Aerospace, Inc.', 'Bombardier Canadair', 'Bombardier Inc', 'Bombardier Learjet Corp.', 'Bombardier, Inc.'], 'Bombardier')

In [47]:
# fix up others
# df['Make'] = df['Make'].replace('', '')

df['Make'] = df['Make'].replace('Britten Norman', 'Britten-Norman')
df['Make'] = df['Make'].replace('Burkhart Grob Flugzeugbau', 'Burkhart Grob')
df['Make'] = df['Make'].replace('Camair Aircraft Corp.', 'Camair')
df['Make'] = df['Make'].replace('Canadair Ltd', 'Canadair')
df['Make'] = df['Make'].replace('C A S A', 'Casa')
df['Make'] = df['Make'].replace(['Cessna Aircraft', 'Cessna Aircraft Company', 'Cessna Robertson', 'Cessna Reims'], 'Cessna')
df['Make'] = df['Make'].replace('Christen Industries Inc', 'Christen Industries')
df['Make'] = df['Make'].replace(['Cirrus Design Corp', 'Cirrus Design Corp.', 'Cirrus Design Corporation'], 'Cirrus')
df['Make'] = df['Make'].replace('Construcciones Aeronauticas Sa', 'Construcciones Aeronauticas')
df['Make'] = df['Make'].replace(['Dassault Aviation', 'Dassault Falcon', 'Dassault-Breguet', 'Dassault/Sud'], 'Dassault')

In [48]:
df['Make'] = df['Make'].replace('Dehavilland', 'De Havilland')
df['Make'] = df['Make'].replace(['Diamond Aircraft Ind Gmbh', 'Diamond Aircraft Ind Inc', 'Diamond Aircraft Industries'], 'Diamond')
df['Make'] = df['Make'].replace('Dornier Gmbh', 'Dornier')
df['Make'] = df['Make'].replace(['Eclipse', 'Eclipse Aviation Corp', 'Eclipse Aviation Corporation'], 'Eclipse Aviation')
df['Make'] = df['Make'].replace(['Embraer S A', 'Embraer S.A.', 'Embraer Sa', 'Embraer Aircraft', 'Embraer Executive Aircraft Inc', 'Embraer-Empresa Brasileira De'], 'Embraer')
df['Make'] = df['Make'].replace('Erickson Air-Crane, Inc.', 'Erickson')
df['Make'] = df['Make'].replace('Extra Flugzeugbau', 'Extra')
df['Make'] = df['Make'].replace(['Eurocopter Deutschland', 'Eurocopter France'], 'Eurocopter')
df['Make'] = df['Make'].replace(['Evektor Aerotechnik', 'Evektor-Aerotechnik As'], 'Evektor')
df['Make'] = df['Make'].replace(['Fairchild Hiller', 'Fairchild Industries', 'Fairchild Merlin', 'Fairchild Swearingen'], 'Fairchild')
df['Make'] = df['Make'].replace('Firefly Balloons', 'Firefly')
df['Make'] = df['Make'].replace('Flight Design Gmbh', 'Flight Design')

In [49]:
df['Make'] = df['Make'].replace('Galaxy Balloons, Inc.', 'Galaxy')
df['Make'] = df['Make'].replace(['Gates Lear Jet', 'Gates Lear Jet Corp.', 'Gates Learjet Corp', 'Gates Learjet Corp.', 'Gates Learjet Corporation'], 'Gates Learjet')
df['Make'] = df['Make'].replace('Glaser-Dirks-Flugzeubau', 'Glaser-Dirks')

In [50]:
df['Make'] = df['Make'].replace('Goodyear Aerospace', 'Goodyear')
df['Make'] = df['Make'].replace(['Grumman Acft Eng Cor-Schweizer', 'Grumman Aircraft Eng Corp', 'Grumman American', 'Grumman American Avn. Corp.', 'Grumman American Corporation'], 'Grumman')
df['Make'] = df['Make'].replace(['Hawker Beech', 'Hawker Beechcraft', 'Hawker Beechcraft Corp', 'Hawker Beechcraft Corp.', 'Hawker Beechcraft Corporation', 'Hawker Siddeley', 'Hawker Siddely', 'Hawker-Beechcraft'], 'Hawker')
df['Make'] = df['Make'].replace(['Honda Aircraft', 'Honda Aircraft Co Llc', 'Honda Jet', 'Howard Aircraft Corp.'], 'Honda')
df['Make'] = df['Make'].replace('Israel Aerospace Industriesltd', 'Israel Aircraft Industries')
df['Make'] = df['Make'].replace('Learjet Inc', 'Learjet')
df['Make'] = df['Make'].replace(['Liberty Aerospace', 'Liberty Aerospace Inc.', 'Liberty Aerospace Incorporate'], 'Liberty')
df['Make'] = df['Make'].replace('Lindstrand Balloons', 'Lindstrand')
df['Make'] = df['Make'].replace('M-Squared Aircraft', 'M-Squared')
df['Make'] = df['Make'].replace('M7 Aerospace Llc', 'M7 Aerospace')
df['Make'] = df['Make'].replace('Martin Company', 'Martin')
df['Make'] = df['Make'].replace('Maule Aircraft Corp', 'Maule')

In [51]:
df['Make'] = df['Make'].replace(['Md Helicopters', 'Md Helicopters, Inc.'], 'Md Helicopter')
df['Make'] = df['Make'].replace('Messerschmitt-Boelkow-Blohm', 'Messerschmitt')
df['Make'] = df['Make'].replace('Mooney Aircraft Corp.', 'Mooney')
df['Make'] = df['Make'].replace('National Ballooning Ltd', 'National Balloon')
df['Make'] = df['Make'].replace(['North American Rockwell', 'North American Rockwell Corp.'], 'North American')
df['Make'] = df['Make'].replace(['Partenavia S.P.A.', 'Partenavia spa'], 'Partenavia')
df['Make'] = df['Make'].replace(['Piaggio Aero Industries S.P.A.', 'Piaggio Aero Industries Spa', 'Piaggio Industrie'], 'Piaggio')
df['Make'] = df['Make'].replace('Pilatus Britten-Norman', 'Pilatus')
df['Make'] = df['Make'].replace(['Piper Aerostar', 'Piper Aircraft', 'Piper Aircraft Corporation', 'Piper Aircraft Inc', 'Piper Aircraft, Inc.', 'Piper-Aerostar'], 'Piper')

In [52]:
df['Make'] = df['Make'].replace('Quest Aircraft Company', 'Quest')
df['Make'] = df['Make'].replace(['Raytheon Aircraft Company', 'Raytheon Corporate Jets', 'Raytheon Corporate Jets Inc'], 'Raytheon')
df['Make'] = df['Make'].replace(['Reims Aviation', 'Reims Cessna'], 'Reims')
df['Make'] = df['Make'].replace(['Robinson Helicopter', 'Robinson Helicopter Company', 'Robinson Helicopters'], 'Robinson')
df['Make'] = df['Make'].replace(['Rockwell Commander', 'Rockwell International'], 'Rockwell')
df['Make'] = df['Make'].replace(['Saab-Scania', 'Saab-Scania Ab (Saab)'], 'Saab')
df['Make'] = df['Make'].replace('Schweitzer', 'Schweizer')
df['Make'] = df['Make'].replace(['Short', 'Short Bros', 'Short Bros.', 'Short Bros. & Harland'], 'Short Brothers')
df['Make'] = df['Make'].replace('Slingsby Aviation Plc', 'Slingsby')
df['Make'] = df['Make'].replace(['Smith Aerostar', 'Smith, Ted Aerostar'], 'Smith')
df['Make'] = df['Make'].replace(['Stol Aircraft', 'Stol Aircraft Corp'], 'Stol')

In [53]:
df['Make'] = df['Make'].replace('Textron Aviation Inc', 'Textron Aviation')
df['Make'] = df['Make'].replace('Thrush Aircraft Inc', 'Thrush')
df['Make'] = df['Make'].replace('United Consultant Corp.', 'United Consultants')
df['Make'] = df['Make'].replace('Wag-Aero', 'Wagaero')
df['Make'] = df['Make'].replace('Weatherly Aviation Co Inc', 'Weatherly')
df['Make'] = df['Make'].replace(['Wing Aircraft', 'Wing Aircraft Co.'], 'Wing')
df['Make'] = df['Make'].replace('Worner Balloon Works', 'Worner')
df['Make'] = df['Make'].replace(['Wsk Pzl Krosno', 'Wsk Pzl-Krosno'], 'Wsk Pzl')

In [54]:
df['Make'].value_counts().sort_index(ascending=True).iloc[300:]

Swearingen                    143
Swift                           1
Szd                             1
Taylorcraft                     7
Tecnam                          2
Ted Smith                       3
Temco                           7
Textron Aviation                7
The Boeing Company              3
Thrush                          4
Thunder And Colt               13
Tl Ultralight Sro               2
Travel Air                      1
Tumbling Goose Llc              1
Tupolev                         3
Turbo Thrush                    1
Twin Commander Acft. Corp.      4
United Consultants              2
Universal                       1
Vans                            6
Velocity                        1
Vickers                         2
Viking Air Limited              1
Vought                          1
Waco                            2
Wag-A-Bond Traveller            1
Wagaero                         2
Wdl                             1
Weatherly                       4
Westland Helic

In [55]:
df['Make'].value_counts()

Cessna                     3093
Piper                      2498
Beech                      2122
Boeing                     1281
Mcdonnell Douglas           455
                           ... 
Czech Sport Aircraft As       1
Wdl                           1
Asta                          1
Vought                        1
Worldwide Aero Corp           1
Name: Make, Length: 341, dtype: int64